In [1]:
import pandas as pd
import numpy as np

from datetime import datetime as dt
from functools import partial, wraps

from plasticc.featurize import process_meta, featurize
from plasticc.training import path_from_cv_score, train_and_validate
from plasticc.final import featurize_test, predict_test

Using TensorFlow backend.


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
fcp = {
    'flux': {
        'longest_strike_above_mean': None,
        'longest_strike_below_mean': None,
        'mean_change': None,
        'mean_abs_change': None,
        'length': None,
    },

    'flux_by_flux_ratio_sq': {
        'longest_strike_above_mean': None,
        'longest_strike_below_mean': None,       
    },

    'flux_passband': {
        'fft_coefficient': [
                {'coeff': 0, 'attr': 'abs'}, 
                {'coeff': 1, 'attr': 'abs'}
            ],
        'kurtosis' : None, 
        'skewness' : None,
    },

    'mjd': {
        'maximum': None, 
        'minimum': None,
        'mean_change': None,
        'mean_abs_change': None,
    },
}

In [4]:
aggs = {
    'flux': ['min', 'max', 'mean', 'median', 'std', 'skew'],
    'flux_err': ['min', 'max', 'mean', 'median', 'std', 'skew'],
    'detected': ['mean'],
    'flux_ratio_sq':['sum', 'skew'],
    'flux_by_flux_ratio_sq':['sum','skew'],
}

### Generate features for model training

In [5]:
%%time
meta_train = process_meta('../data/raw/training_set_metadata.csv')
train = pd.read_csv('../data/raw/training_set.csv')

CPU times: user 2.7 s, sys: 328 ms, total: 3.03 s
Wall time: 1.77 s


In [6]:
%%time
X = featurize(train, meta_train, aggs, fcp, n_jobs=4)

Feature Extraction: 100%|██████████| 20/20 [00:01<00:00, 19.84it/s]


CPU times: user 1min 10s, sys: 3.01 s, total: 1min 13s
Wall time: 32.3 s


In [7]:
if 'target' in X:
    y = X['target']
    del X['target']
else:
    print("What the duck")
    3//0

In [8]:
%%time
train_mean = X.mean(axis=0)
#train_mean.to_hdf('train_data.hdf5', 'data')
# pd.set_option('display.max_rows', 500)
#import pdb; pdb.set_trace()
X.fillna(0, inplace=True)

CPU times: user 144 ms, sys: 0 ns, total: 144 ms
Wall time: 7.16 ms


### Train model with CV

In [18]:
lgbm_params = {
    'device': 'cpu',
    'objective': 'multiclass',
    'num_class': 14,
    'boosting_type': 'gbdt',
    'n_jobs': 16,
    'max_depth': 6,
    'n_estimators': 1024,
    'subsample_freq': 2,
    'subsample_for_bin': 5000,
    'min_data_per_group': 100,
    'max_cat_to_onehot': 4,
    'cat_l2': 1.0,
    'cat_smooth': 59.5,
    'max_cat_threshold': 32,
    'metric_freq': 10,
    'verbosity': -1,
    'metric': 'multi_logloss',
    'xgboost_dart_mode': False,
    'uniform_drop': False,
    'colsample_bytree': 0.5,
    'drop_rate': 0.173,
    'learning_rate': 0.0267,
    'max_drop': 5,
    'min_child_samples': 10,
    'min_child_weight': 100.0,
    'min_split_gain': 0.126,
    'num_leaves': 7,
    'reg_alpha': 0.1,
    'reg_lambda': 0.00023,
    'skip_drop': 0.44,
    'subsample': 0.75,
    'max_bin': 32,
    'min_data_in_leaf': 13,
    'lambda_l1': 2,
}


xgb_params = {
    'objective': 'multiclass',
    'booster': 'gbdtree',
    'n_jobs': 16,
    'max_depth': 7,
    'n_estimators': 1024,
    'verbosity': -1,
    'colsample_bytree': 0.5,
    'learning_rate': 0.0267,
    'min_child_weight': 100.0,
    'reg_alpha': 0.1,
    'reg_lambda': 0.00023,
    'subsample': 0.75
}

In [19]:
colnames_to_ignore = set([
    'object_id',
    'hostgal_specz',
    'ra',
    'decl',
    'gal_l',
    'gal_b',
    'ddf',
    'latlon1',
    'haversine',
])
feature_colnames = [col for col in X.columns if col not in colnames_to_ignore]
id_colname = 'object_id'

In [20]:
len(feature_colnames)

60

In [21]:
%%time
# modeling from CV
clfs, score, importances = train_and_validate(
    X=X, 
    y=y, 
    feature_colnames=feature_colnames, 
    id_colname=id_colname, 
    model='lgbm', 
    model_params=lgbm_params, 
    nr_fold=6, 
    random_state=1
)

Training until validation scores don't improve for 50 rounds.
[100]	training's multi_logloss: 0.340002	training's wloss: 0.8295	valid_1's multi_logloss: 1.72625	valid_1's wloss: 1.02895
[200]	training's multi_logloss: 0.182281	training's wloss: 0.552799	valid_1's multi_logloss: 1.35969	valid_1's wloss: 0.774142
[300]	training's multi_logloss: 0.135836	training's wloss: 0.44372	valid_1's multi_logloss: 1.19622	valid_1's wloss: 0.705004
[400]	training's multi_logloss: 0.112114	training's wloss: 0.380978	valid_1's multi_logloss: 1.10718	valid_1's wloss: 0.681572
[500]	training's multi_logloss: 0.0965806	training's wloss: 0.337432	valid_1's multi_logloss: 1.0476	valid_1's wloss: 0.6709
[600]	training's multi_logloss: 0.0850397	training's wloss: 0.303567	valid_1's multi_logloss: 0.999548	valid_1's wloss: 0.666795
Early stopping, best iteration is:
[612]	training's multi_logloss: 0.0838321	training's wloss: 0.299934	valid_1's multi_logloss: 0.994546	valid_1's wloss: 0.666546
no 1-fold loss: 

In [22]:
importances.sort_values(by='mean_gain', ascending=False).head(15)

,feature,gain,fold,mean_gain
54,mjd_diff_det,2485,5,2623.500000
54,mjd_diff_det,2308,2,2623.500000
54,mjd_diff_det,2845,3,2623.500000
54,mjd_diff_det,2656,4,2623.500000
54,mjd_diff_det,3017,6,2623.500000
54,mjd_diff_det,2430,1,2623.500000
57,distmod,2692,6,2182.833333
57,distmod,2050,5,2182.833333
57,distmod,2085,4,2182.833333
57,distmod,2461,3,2182.833333


In [23]:
submission_file_path = path_from_cv_score(score)
submission_file_path

'/home/kk385830/astronomical-classification/submissions/subm_0.633887_2018-12-12-20-10.csv'

### Test set features

In [14]:
chunk_size_one_100 = 453653105 // 100 + 1
# chunk_size_one_100 = 1000001 // 100 + 1
chunk_size_one_100

4536532

In [ ]:
%%time
# should take 100x (time after 1st iteration)
featurize_test(
    featurize_configs={'aggs': aggs, 'fcp': fcp}, 
    n_jobs=12,
    meta_path='../data/raw/test_set_metadata.csv',
    test_path='../data/raw/test_set.csv',
    output_path='../data/features/test-all-feat-from-kernel-repro.csv',
    id_colname='object_id',
    chunks=chunk_size_one_100,  # alternatively: 5000000
)

Feature Extraction: 100%|██████████| 60/60 [00:01<00:00, 37.97it/s]


        4536532 done in   1.0 minutes


Feature Extraction: 100%|██████████| 60/60 [00:01<00:00, 38.33it/s]


        9073064 done in   1.9 minutes


Feature Extraction: 100%|██████████| 60/60 [00:03<00:00, 18.97it/s]


...

      367459092 done in 146.8 minutes


Feature Extraction: 100%|██████████| 60/60 [00:03<00:00, 15.05it/s]


      371995624 done in 148.6 minutes


Feature Extraction: 100%|██████████| 60/60 [00:03<00:00, 17.05it/s]


      376532156 done in 150.3 minutes


### Test set predictions

In [24]:
%%time
submission = predict_test(
    clfs=clfs, 
    feature_colnames=feature_colnames, 
    id_colname=id_colname, 
    input_path='../data/features/test-all-feat-from-kernel-repro.csv', 
    output_path=submission_file_path, 
    verbose=True
)

Loading data...


  0%|          | 0/6 [00:00<?, ?it/s]

Generating predictions...


100%|██████████| 6/6 [12:49<00:00, 132.60s/it]


Postprocessing...
Submission shape before grouping: (3492890, 16)
Submission shape after grouping: (3492890, 15)
Submission shape after postprocessing: (3492890, 15)
Validating submission file...
Saving submission...
Submission saved to f/home/kk385830/astronomical-classification/submissions/subm_0.633887_2018-12-12-20-10.csv
CPU times: user 3h 19min 16s, sys: 1min 48s, total: 3h 21min 4s
Wall time: 15min 27s


In [25]:
submission.head()

,class_6,class_15,class_16,class_42,class_52,class_53,class_62,class_64,class_65,class_67,class_88,class_90,class_92,class_95,class_99
object_id,,,,,,,,,,,,,,,
13,0.000039,0.000657,0.000029,0.366398,0.368153,0.000064,0.084674,0.000019,0.000019,0.000734,0.000127,0.038328,0.000037,0.000157,0.140565
14,0.000054,0.005733,0.000066,0.099824,0.029541,0.000090,0.055105,0.000179,0.000440,0.015483,0.000307,0.676560,0.000065,0.016401,0.100153
17,0.000124,0.007659,0.000180,0.069897,0.100743,0.000183,0.025223,0.006217,0.000202,0.063850,0.003200,0.592817,0.000065,0.010686,0.118956
23,0.000117,0.001379,0.000108,0.025240,0.011016,0.000160,0.036232,0.005721,0.000174,0.386109,0.000227,0.274889,0.000056,0.113153,0.145418
34,0.000030,0.001124,0.000026,0.072985,0.139158,0.000052,0.023494,0.000035,0.000053,0.007502,0.000046,0.649205,0.000021,0.000144,0.106124
